In [1]:
import numpy as np
import torch
import gym
import argparse
import os

import utils
# import OurDDPG
# import DDPG
from air_hockey_challenge.framework.air_hockey_challenge_wrapper import AirHockeyChallengeWrapper
from air_hockey_agent.agent_builder import build_agent
from air_hockey_challenge.utils.kinematics import inverse_kinematics, jacobian, forward_kinematics

ModuleNotFoundError: No module named 'utils'

In [10]:

# # Runs policy for X episodes and returns average reward
# # A fixed seed is used for the eval environment
# def eval_policy(policy, eval_episodes=10):
# 	eval_env = AirHockeyChallengeWrapper(env="3dof-hit", action_type="position-velocity", interpolation_order=3, debug=True,custom_reward_function=custom_rewards)


# 	avg_reward = 0.
# 	for _ in range(eval_episodes):
# 		state, done = eval_env.reset(), False
# 		while not done:
# 			action = policy.draw_action(np.array(state)).reshape(2,3)
# 			state, reward, done, _ = eval_env.step(action)
# 			avg_reward += reward

# 	avg_reward /= eval_episodes

# 	print("---------------------------------------")
# 	print(f"Evaluation over {eval_episodes} episodes: {avg_reward:.3f}")
# 	print("---------------------------------------")
# 	return avg_reward



In [11]:
env.env_info

{'table': {'length': 1.948, 'width': 1.038, 'goal_width': 0.25},
 'puck': {'radius': 0.03165},
 'mallet': {'radius': 0.04815},
 'n_agents': 2,
 'robot': {'n_joints': 7,
  'ee_desired_height': 0.1645,
  'joint_vel_limit': array([[-1.48352986, -1.48352986, -1.74532925, -1.30899694, -2.26892803,
          -2.35619449, -2.35619449],
         [ 1.48352986,  1.48352986,  1.74532925,  1.30899694,  2.26892803,
           2.35619449,  2.35619449]]),
  'joint_acc_limit': array([[-14.83529864, -14.83529864, -17.45329252, -13.08996939,
          -22.68928028, -23.5619449 , -23.5619449 ],
         [ 14.83529864,  14.83529864,  17.45329252,  13.08996939,
           22.68928028,  23.5619449 ,  23.5619449 ]]),
  'base_frame': [array([[ 1.  ,  0.  ,  0.  , -1.51],
          [ 0.  ,  1.  ,  0.  ,  0.  ],
          [ 0.  ,  0.  ,  1.  , -0.1 ],
          [ 0.  ,  0.  ,  0.  ,  1.  ]]),
   array([[-1.  ,  0.  ,  0.  ,  1.51],
          [ 0.  , -1.  ,  0.  ,  0.  ],
          [ 0.  ,  0.  ,  1.  , -0.1 ],


In [12]:
env = AirHockeyChallengeWrapper(env="7dof-hit", interpolation_order=1, debug=True)
policy = build_agent(env.env_info)
# evaluations = eval_policy(policy)

In [13]:
# env = AirHockeyChallengeWrapper(env="3dof-hit", action_type="position-velocity", interpolation_order=3, debug=True)
from air_hockey_challenge.utils.kinematics import forward_kinematics, jacobian
state, done = env.reset(), False
env.render()

AttributeError: 'numpy.ndarray' object has no attribute 'is_scalar'

In [24]:
policy = build_agent(env.env_info) 

In [25]:
from casadi import SX, sin, Function, inf,vertcat,nlpsol,qpsol,sumsqr
import numpy as np

In [40]:
def integrate_RK4(s_expr, a_expr, sdot_expr, dt, N_steps=1):
    '''RK4 integrator.

    s_expr, a_expr: casadi expression that have been used to define the dynamics sdot_expr
    sdot_expr:      casadi expr defining the rhs of the ode
    dt:             integration interval
    N_steps:        number of integration steps per integration interval, default:1
    '''

    h = dt/N_steps

    s_end = s_expr

    sdot_fun = Function('xdot', [s_expr, a_expr], [sdot_expr])

    for _ in range(N_steps):

    # FILL IN YOUR CODE HERE
        v_1 = sdot_fun(s_end, a_expr)
        v_2 = sdot_fun(s_end + 0.5 * h * v_1, a_expr)
        v_3 = sdot_fun(s_end + 0.5 * h * v_2, a_expr)
        v_4 = sdot_fun(s_end + v_3 * h, a_expr)
        s_end += (1/6) * (v_1 + 2 * v_2 + 2 * v_3 + v_4) * h

    F_expr = s_end

    return F_expr

In [41]:
def solve_casadi(x0_bar,x_des,jac):
    # continuous model dynamics
    n_s = 3  # number of states
    n_a = 7  # number of actions

    x = SX.sym('x')
    y = SX.sym('y')
    z = SX.sym('z')

    omega = SX.sym('omega',7)

    s = vertcat(x,y,z)
    # q_0 = policy.robot_data.qpos.copy()
    # jac = jacobian(policy.robot_model, policy.robot_data,q_0)[:3, :7]
    s_dot = vertcat(jac @ omega)
    # Define number of steps in the control horizon and discretization step
    # print(s_dot)
    N = 10
    delta_t = 1/50
    # Define RK4 integrator function and initial state x0_bar
    F_rk4 = Function("F_rk4", [s, omega], [integrate_RK4(s, omega, s_dot, delta_t)])
    # x0_bar = [-.5, .5,.165]

    # Define the weighting matrix for the cost function
    Q = np.eye(n_s)
    R = np.eye(n_a)

        # Start with an empty NLP
    w = []
    w0 = []
    lbw = []
    ubw = []
    J = 0
    g = []
    lbg = []
    ubg = []

    # "Lift" initial conditions
    Xk = SX.sym('X0', 3)
    w += [Xk]
    lbw += x0_bar    # set initial state
    ubw += x0_bar    # set initial state
    w0 += x0_bar     # set initial state

    # Formulate the NLP
    for k in range(N):
        # New NLP variable for the control
        Uk = SX.sym('U_' + str(k),7)
        w   += [Uk]
        lbw += [-1.48352986, -1.48352986, -1.74532925, -1.30899694, -2.26892803,
        -2.35619449, -2.35619449]
        ubw += [1.48352986, 1.48352986, 1.74532925, 1.30899694, 2.26892803,
        2.35619449, 2.35619449]
        w0  += [0,0,0,0,0,0,0]

        # Integrate till the end of the interval
        Xk_end = F_rk4(Xk, Uk)
        # J = J + delta_t *(sumsqr((Xk-x_des).T @ Q )+ sumsqr(R@Uk)) # Complete with the stage cost
        J = J + (sumsqr((Xk-x_des))) # Complete with the stage cost

        # New NLP variable for state at end of interval
        Xk = SX.sym(f'X_{k+1}', 3)
        w += [Xk]
        lbw += [.5,-.5,0.165]
        ubw += [1.5,.5,0.25]
        w0 += [0, 0,0.25]

        # Add equality constraint to "close the gap" for multiple shooting
        g   += [Xk_end-Xk]
        lbg += [0, 0,0]
        ubg += [0, 0,0]
    J = J + sumsqr((Xk-x_des)) # Complete with the terminal cost (NOTE it should be weighted by delta_t)

    # Create an NLP solver
    prob = {'f': J, 'x': vertcat(*w), 'g': vertcat(*g)}
    solver = nlpsol('solver', 'ipopt', prob)

    # Solve the NLP
    sol = solver(x0=w0, lbx=lbw, ubx=ubw, lbg=lbg, ubg=ubg)
    w_opt = sol['x'].full().flatten()
    # return np.array([w_opt[3::10],w_opt[4::10],w_opt[5::10],w_opt[6::10],w_opt[7::10],w_opt[8::10],w_opt[9::10]])
    
    return np.array(w_opt[3:10]),np.array(w_opt[-4:]),solver.stats()['success']
    # return solver

In [42]:
state, done = env.reset(), False
env.render()

AttributeError: 'numpy.ndarray' object has no attribute 'is_scalar'

In [29]:
q_0 = state[6:13]
jac = jacobian(policy.robot_model, policy.robot_data,q_0)[:3, :7]
x0 = list(forward_kinematics(policy.robot_model, policy.robot_data, q_0)[0])
# x0 = [ 6.50000005e-01, -1.01327613e-38,  1.64500044e-01]
print(x0)

NameError: name 'state' is not defined

In [30]:
np.cos(0.5*2*np.pi*/100)

SyntaxError: invalid syntax (629198716.py, line 1)

In [1]:
for i in range(100):
    x_des = np.array([np.sin(0.25*2*np.pi*i/100),np.cos(0.25*2*np.pi*i/100),0.165]) + np.array([0.5,0,0])
    jac = jacobian(policy.robot_model, policy.robot_data,q_0)[:3, :7]
    x0 = list(forward_kinematics(policy.robot_model, policy.robot_data, q_0)[0])
    # x0 = [ 6.50000005e-01, -1.01327613e-38,  1.64500044e-01]
    print(x0)
    # x_des = [1.2,0.02,0.165]    
    print(x_des)
    q,_ = solve_casadi(x0,x_des,jac)
    # print(q)
    next_q = q_0 + q*0.02
    next_state, reward, done, info = env.step(next_q)
    env.render()
    q_0 = next_state[6:13]

NameError: name 'np' is not defined

In [151]:

jac = jacobian(policy.robot_model, policy.robot_data,q_0)[:3, :7]
x0 = list(forward_kinematics(policy.robot_model, policy.robot_data, q_0)[0])
# x0 = [ 6.50000005e-01, -1.01327613e-38,  1.64500044e-01]
print(x0)
x_des = [1.2,0.02,0.165]
print(x_des)
q,_ = solve_casadi(x0,x_des,jac)
print(_)
# print(q)
# next_q = q_0 + q*0.02
# next_state, reward, done, info = env.step(next_q)
# env.render()
# q_0 = next_state[6:13]

[1.4562590692564716, 0.0006842168974050397, 0.14155115716534322]
[1.2, 0.02, 0.165]
This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      257
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       30

Total number of variables............................:      100
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      100
                     variables with only upper bounds:        0
Total number of equality constraints.................:       30
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_

ValueError: too many values to unpack (expected 2)

In [147]:
q.stats()['success']

True

In [13]:
print(_)

[-2.35619449  0.68        0.02        0.165     ]


In [14]:
print(q)


[ 1.48352986  1.48352986  1.74532925  0.4550553  -1.83892528 -2.35619449
 -2.35619449]


In [15]:
next_state, reward, done, info = env.step(next_q)
env.render()

In [16]:
# # Define number of steps in the control horizon and discretization step
# N = 20
# delta_t = 1/50
# # Define RK4 integrator function and initial state x0_bar
# F_rk4 = Function("F_rk4", [s, omega], [integrate_RK4(s, omega, s_dot, delta_t)])
# x0_bar = [-.5, .5,.165]

# # Define the weighting matrix for the cost function
# Q = np.eye(n_s)
# R = np.eye(n_a)

In [17]:
# # Start with an empty NLP
# w = []
# w0 = []
# lbw = []
# ubw = []
# J = 0
# g = []
# lbg = []
# ubg = []

# # "Lift" initial conditions
# Xk = SX.sym('X0', 3)
# w += [Xk]
# lbw += x0_bar    # set initial state
# ubw += x0_bar    # set initial state
# w0 += x0_bar     # set initial state

# # Formulate the NLP
# for k in range(N):
#     # New NLP variable for the control
#     Uk = SX.sym('U_' + str(k),7)
#     w   += [Uk]
#     lbw += [-1.48352986, -1.48352986, -1.74532925, -1.30899694, -2.26892803,
#        -2.35619449, -2.35619449]
#     ubw += [1.48352986, 1.48352986, 1.74532925, 1.30899694, 2.26892803,
#        2.35619449, 2.35619449]
#     w0  += [0,0,0,0,0,0,0]

#     # Integrate till the end of the interval
#     Xk_end = F_rk4(Xk, Uk)
#     J = J + sumsqr(delta_t * 1/2 * (Xk.T @ Q @ Xk + R * Uk)) # Complete with the stage cost

#     # New NLP variable for state at end of interval
#     Xk = SX.sym(f'X_{k+1}', 3)
#     w += [Xk]
#     lbw += [-.5,-.5,0.165]
#     ubw += [0,.5,0.175]
#     w0 += [0, 0,0]

#     # Add equality constraint to "close the gap" for multiple shooting
#     g   += [Xk_end-Xk]
#     lbg += [0, 0,0]
#     ubg += [0, 0,0]
# J = J + sumsqr(1/2 * (Xk.T @ Q @ Xk)) # Complete with the terminal cost (NOTE it should be weighted by delta_t)

# # Create an NLP solver
# prob = {'f': J, 'x': vertcat(*w), 'g': vertcat(*g)}
# solver = qpsol('solver', 'qpoases', prob)

# # Solve the NLP
# sol = solver(x0=w0, lbx=lbw, ubx=ubw, lbg=lbg, ubg=ubg)
# w_opt = sol['x'].full().flatten()

In [18]:
s_traj_opt = np.vstack((w_opt[::3], w_opt[1::3]))
a_traj_opt = w_opt[2::3][np.newaxis, ...]

NameError: name 'w_opt' is not defined

In [ ]:
sol['x'][1].shape

(1, 1)

In [ ]:
policy.env_info['robot']['joint_vel_limit'][1]

array([1.48352986, 1.48352986, 1.74532925, 1.30899694, 2.26892803,
       2.35619449, 2.35619449])

In [ ]:
import numpy as np
from casadi import SX, vertcat, Function, sumsqr

P = SX.sym('P',3)
V = SX.sym('V',3)

des_state = [1,1,0.1645]

X = vertcat(P, V)

theta = SX.sym('theta',7)
theta_dot = SX.sym('theta_dot',7)

u = vertcat(theta, theta_dot)

# p,r = forward_kinematics(policy.robot_model, policy.robot_data,des_pos)
F_forward = Function("F_forward", [theta],[frwd_kinematics(theta)])

f = sumsqr(P-des_state)
g = P[2]-0.1645

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/captain/anaconda3/envs/challenge/lib/python3.8/site-packages/casadi/casadi.py", line 13398, in __array__
    return self.full()
  File "/home/captain/anaconda3/envs/challenge/lib/python3.8/site-packages/casadi/casadi.py", line 11693, in <lambda>
    __getattr__ = lambda self, name: _swig_getattr(self, SX, name)
  File "/home/captain/anaconda3/envs/challenge/lib/python3.8/site-packages/casadi/casadi.py", line 83, in _swig_getattr
    raise AttributeError("'%s' object has no attribute '%s'" % (class_type.__name__, name))
AttributeError: 'SX' object has no attribute 'full'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/captain/anaconda3/envs/challenge/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_42207/749874423.py", line 17, in <module>
    F_forward = F

In [ ]:
# Start with an empty NLP
w = []
w0 = []
lbw = []
ubw = []
J = 0
g = []
lbg = []
ubg = []

# "Lift" initial conditions
Xk = SX.sym('X0', 2)
w += [Xk]
lbw += x0_bar    # set initial state
ubw += x0_bar    # set initial state
w0 += x0_bar     # set initial state

# Formulate the NLP
for k in range(N):
    # New NLP variable for the control
    Uk = SX.sym('U_' + str(k))
    w   += [Uk]
    lbw += [-1]
    ubw += [1]
    w0  += [0]

    # Integrate till the end of the interval
    Xk_end = F_rk4(Xk, Uk)
    J = J + delta_t * 1/2 * (Xk.T @ Q @ Xk + R * Uk**2) # Complete with the stage cost

    # New NLP variable for state at end of interval
    Xk = SX.sym(f'X_{k+1}', 2)
    w += [Xk]
    lbw += [-np.pi/2, -inf]
    ubw += [2*np.pi, inf]
    w0 += [0, 0]

    # Add equality constraint to "close the gap" for multiple shooting
    g   += [Xk_end-Xk]
    lbg += [0, 0]
    ubg += [0, 0]
J = J + 1/2 * (Xk.T @ Q @ Xk) # Complete with the terminal cost (NOTE it should be weighted by delta_t)

# Create an NLP solver
prob = {'f': J, 'x': vertcat(*w), 'g': vertcat(*g)}
solver = nlpsol('solver', 'ipopt', prob)

# Solve the NLP
sol = solver(x0=w0, lbx=lbw, ubx=ubw, lbg=lbg, ubg=ubg)
w_opt = sol['x'].full().flatten()

SX([theta_0, theta_1, theta_2, theta_3, theta_4, theta_5, theta_6])

In [ ]:
# Define number of steps in the control horizon and discretization step
N = 2
delta_t = 1/50
# Define RK4 integrator function and initial state x0_bar
x0_bar = [des_current_pos, des_current_vel]

# Define the weighting matrix for the cost function
# Q = np.eye(6)
# R = np.eye(n_a)

In [ ]:
# Start with an empty NLP
w = []
w0 = []
lbw = []
ubw = []
J = 0
g = []
lbg = []
ubg = []

# "Lift" initial conditions
Xk = SX.sym('X0', )
w += [Xk]
lbw += x0_bar    # set initial state
ubw += x0_bar    # set initial state
w0 += x0_bar     # set initial state

# Formulate the NLP
for k in range(N):
    # New NLP variable for the control
    Uk = SX.sym('U_' + str(k))
    w   += [Uk]
    lbw += [-1]
    ubw += [1]
    w0  += [0]

    # Integrate till the end of the interval
    Xk_end = F_rk4(Xk, Uk)
    J = J + delta_t * 1/2 * (Xk.T @ Q @ Xk + R * Uk**2) # Complete with the stage cost

    # New NLP variable for state at end of interval
    Xk = SX.sym(f'X_{k+1}', 2)
    w += [Xk]
    lbw += [-np.pi/2, -inf]
    ubw += [2*np.pi, inf]
    w0 += [0, 0]

    # Add equality constraint to "close the gap" for multiple shooting
    g   += [Xk_end-Xk]
    lbg += [0, 0]
    ubg += [0, 0]
J = J + 1/2 * (Xk.T @ Q @ Xk) # Complete with the terminal cost (NOTE it should be weighted by delta_t)

# Create an NLP solver
prob = {'f': J, 'x': vertcat(*w), 'g': vertcat(*g)}
solver = nlpsol('solver', 'ipopt', prob)

# Solve the NLP
sol = solver(x0=w0, lbx=lbw, ubx=ubw, lbg=lbg, ubg=ubg)
w_opt = sol['x'].full().flatten()